# Sense from Numbers

## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [1]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")

interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [2]:
import $ivy.`edu.holycross.shot::ohco2:10.16.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:5.2.2`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`

import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
// init(offline=true)

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                              

import $ivy.$                              

import $ivy.$                                          

import $ivy.$                                  

import $ivy.$                                        

import $ivy.$                              


import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
// init(offline=true)

// restrict the output height to avoid scrolling in output cells


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [3]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source

import java.text.SimpleDateFormat
import java.util.Date


import edu.holycross.shot.cite._

import edu.holycross.shot.ohco2._

import edu.holycross.shot.scm._

import edu.holycross.shot.citeobj._

import edu.holycross.shot.citerelation._

import edu.holycross.shot.dse._

import edu.holycross.shot.citebinaryimage._

import edu.holycross.shot.ohco2._


import almond.display.UpdatableDisplay

import almond.interpreter.api.DisplayData.ContentType

import almond.interpreter.api.{DisplayData, OutputHandler}


import java.io.File

import java.io.PrintWriter


import scala.io.Source


import java.text.SimpleDateFormat

import java.util.Date


## Useful Functions

Save a string to a names file:

In [4]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

defined function saveString

Convert a Roman Numeral to an Integer:

In [5]:
def fromRoman(s: String) : Int = {
	try {
		val numerals = Map('I' -> 1, 'V' -> 5, 'X' -> 10, 'L' -> 50, 'C' -> 100, 'D' -> 500, 'M' -> 1000)

		s.toUpperCase.map(numerals).foldLeft((0,0)) {
		  case ((sum, last), curr) =>  (sum + curr + (if (last < curr) -2*last else 0), curr) }._1
	} catch {
		case e:Exception => throw new Exception(s""" "${s}" is not a valid Roman Numeral.""")
	}
}

defined function fromRoman

Like `.split`, but preserving the character we split on:

In [6]:
def splitWithSplitter(text: String, puncs: String): Vector[String] = {
	//val regexWithSplitter = s"((?<=${puncs})|(?=${puncs}))"
    val regexWithSplitter = s"((?<=${puncs}))"
	text.split(regexWithSplitter).toVector.filter(_.size > 0)
}

defined function splitWithSplitter

Pretty Print Things:

In [7]:
def showMe(v:Any):Unit = {
  v match {
    case _:StringHistogram => {
        for ( h <- v.asInstanceOf[StringHistogram].histogram ) {
            println(s"${h.count}\t${h.s}")
        }
    }
  	case _:Corpus => {
  		for ( n <- v.asInstanceOf[Corpus].nodes) {
  			println(s"${n.urn.passageComponent}\t\t${n.text}")
  		}	
  	}
    case _:Vector[Any] => println(s"""\n----\n${v.asInstanceOf[Vector[Any]].mkString("\n")}\n----\n""")
    case _:Iterable[Any] => println(s"""\n----\n${v.asInstanceOf[Iterable[Any]].mkString("\n")}\n----\n""")
    case _ => println(s"\n-----\n${v}\n----\n")
  }
}

defined function showMe

Date Stuff

## Load Some Data

Load up-to-date data from the NY Times. Source: <https://github.com/nytimes/covid-19-data>.

In [8]:
val dataLines: Vector[String] = {
    scala.io.Source.fromURL("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv").mkString.split("\n").toVector
}

// quick test
val badLines = dataLines.filter( l => {
    l.split(",").size != 5
})

assert ( badLines.size == 0 )

dataLines: Vector[String] = Vector(
  "date,state,fips,cases,deaths",
...
badLines: Vector[String] = Vector()

## Make Data Structures

For State data:

In [9]:
case class StateDatum( date: String, state: String, cases: Int, deaths: Int)

defined class StateDatum

For Aggregate data:

In [10]:
case class RunningTally( days: Vector[StateDatum] )

defined class RunningTally

A daily snapshot (for the USA or a single state)

In [11]:
case class DailySnapshot( date: String, newCases: Int, newDeaths: Int)

defined class DailySnapshot

## Load Data

In [12]:

val vst: Vector[StateDatum] = dataLines.tail.map( dl => {
    val fields: Vector[String] = dl.split(",").toVector
    val date: String = fields(0)
   val state: String = fields(1)
    val cases: Int = fields(3).toInt
    val deaths: Int = fields(4).toInt
    StateDatum(date, state, cases, deaths)
})

val rt = RunningTally(vst)


vst: Vector[StateDatum] = Vector(
  StateDatum("2020-01-21", "Washington", 1, 0),
...
rt: RunningTally = RunningTally(
  Vector(
...

## Comparative Data

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

Source for South Carolina Data: [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm)

In [13]:
val usa2017TotalDeaths: Int = 2813502
val usa2017DailyDeaths: Int = usa2017TotalDeaths / 365

// Chronic lower respiratory diseases
val usa2017TotalDeathsCLRD: Int = 160201
val usa2017DailyDeathsCLRD: Int = 160201 / 365

// Influenza and Pneumonia
val usa2017TotalDeathsIPn: Int = 55672
val usa2017DailyDeathsIPn: Int = 55672 / 365

// All respiratory-related deaths
val usa2017TotalDeathsResp: Int = usa2017TotalDeathsCLRD + usa2017TotalDeathsIPn
val usa2017DailyDeathsResp: Int = usa2017TotalDeathsResp / 365

// All respiratory-related deaths, 6-month Flu Season
val usa2017DailyDeathsResp6MoFlu: Int = {
    usa2017DailyDeathsCLRD + (usa2017TotalDeathsIPn / 182 )
}

// Suicide
val usa2017SuicideTotal: Int = 47173
val usa2017SuicideDaily: Int = usa2017SuicideTotal / 365

// Heart Disease
val usa2017HeartTotal: Int = 647457
val usa2017HeartDaily: Int = usa2017HeartTotal / 365

// Accidents
val usa2017AccidentTotal: Int = 169936
val usa2017AccidentDaily: Int = usa2017AccidentTotal / 365

// Cancer
val usa2017CancerTotal: Int = 599108
val usa2017CancerDaily: Int = usa2017CancerTotal / 365



usa2017TotalDeaths: Int = 2813502
usa2017DailyDeaths: Int = 7708
usa2017TotalDeathsCLRD: Int = 160201
usa2017DailyDeathsCLRD: Int = 438
usa2017TotalDeathsIPn: Int = 55672
usa2017DailyDeathsIPn: Int = 152
usa2017TotalDeathsResp: Int = 215873
usa2017DailyDeathsResp: Int = 591
usa2017DailyDeathsResp6MoFlu: Int = 743
usa2017SuicideTotal: Int = 47173
usa2017SuicideDaily: Int = 129
usa2017HeartTotal: Int = 647457
usa2017HeartDaily: Int = 1773
usa2017AccidentTotal: Int = 169936
usa2017AccidentDaily: Int = 465
usa2017CancerTotal: Int = 599108
usa2017CancerDaily: Int = 1641

## Data Functions

In [14]:
def totalNewDaily(data: Vector[StateDatum] = rt.days, state: Option[String] = None): Vector[DailySnapshot] = {
    val sortedData: Vector[StateDatum] = {
        val filtered: Vector[StateDatum] = {
            state match {
                case Some(s) => data.filter(_.state == s)
                case None => {
                    val groupedByDay: Vector[(String, Vector[StateDatum])] = {
                        data.groupBy(_.date).toVector
                    }
                    val merged: Vector[StateDatum] = groupedByDay.map( gbd => {
                        val vec = gbd._2
                        val vecDate: String = vec.head.date
                        val vecState: String = "USA"
                        val vecCases: Int = vec.map(_.cases).sum
                        val vecDeaths: Int = vec.map(_.deaths).sum
                        StateDatum(vecDate, vecState, vecCases, vecDeaths)
                    })
                    merged
                }
            }
        }
        val sorted: Vector[StateDatum] = filtered.sortBy(_.date)
        sorted
    }
    // We don't want a running, cumulative tally, but new cases/deaths each day
    sortedData.zipWithIndex.map( sd => {
        val d: StateDatum = sd._1
        val i: Int = sd._2
        val newCases: Int = {
            if (i == 0) d.cases
            else {
                val totalToday: Int = d.cases
                val totalPrev: Int = {
                    sortedData(i-1).cases
                }
                totalToday - totalPrev
            }
        }
        val newDeaths: Int = {
            if (i == 0) d.deaths
            else {
                val totalToday: Int = d.deaths
                val totalPrev: Int = {
                    sortedData(i-1).deaths
                }
                totalToday - totalPrev
            }
        }
        DailySnapshot(d.date, newCases, newDeaths)
    })
    
}

defined function totalNewDaily

## The Pure “Curve”

Just 2020 COVID-19 Deaths, as reported by the New York Times (<https://github.com/nytimes/covid-19-data>).

For comparison, we also show New York only, and the national total with New York removed.

In [15]:
val deaths: Seq[Int] = totalNewDaily(state = None).map(_.newDeaths).toSeq

val nyDeaths: Seq[Int] = totalNewDaily(state = Some("New York")).map(_.newDeaths).toSeq

val aligner: Seq[Int] = {
    val diff: Int = deaths.size - nyDeaths.size
    val padding: Seq[Int] = (1 to diff).map(n => 0).toSeq
    padding
}

nyDeaths
aligner
deaths.size
(aligner ++ nyDeaths).toSeq.size

val paddedNy: Seq[Int] = (aligner ++ nyDeaths).toVector
paddedNy


//val nyDeaths: Seq[Int] = (aligner ++ nyDeathsShort).toSeq

val howManyDays: Int = deaths.size
val ipnBase: Seq[Int] = (1 to howManyDays).map( d => usa2017DailyDeathsResp)

// Let's remove NY
val noNy: Seq[Int] = deaths.zip(paddedNy).map( t => {
    t._1 - t._2
})




val trace3 = Scatter(
  (1 to 35),
  deaths.takeRight(35),
  name = "2020 Covid Daily Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val trace4 = Scatter(
  (1 to 35),
  paddedNy.takeRight(35),
  name = "New York Covid Daily Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 0, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 0, 0, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val trace5 = Scatter(
  (1 to 35),
  noNy.takeRight(35),
  name = "New York Removed",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 204, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 0, 0, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(trace3, trace4, trace5)

val layout = Layout("Daily USA COVID Deaths, Past 35 Days")

plot(data, layout)

deaths: Seq[Int] = Vector(
  0,
...
nyDeaths: Seq[Int] = Vector(
  0,
...
aligner: Seq[Int] = Vector(
  0,
...
res14_3: Seq[Int] = Vector(
  0,
...
res14_4: Seq[Int] = Vector(
  0,
...
res14_5: Int = 78
res14_6: Int = 78
paddedNy: Seq[Int] = Vector(
  0,
...
res14_8: Seq[Int] = Vector(
  0,
...
howManyDays: Int = 78
ipnBase: Seq[Int] = Vector(
  591,
...
noNy: Seq[Int] = Vector(
  0,
...
trace3: Scatter = Scatter(
  Some(
...
trace4: Scatter = Scatter(
  Some(
...
trace5: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Daily USA COVID Deaths, Past 35 Days"),
...
res14_17: String = "plot-118ce446-d0fa-4217-8210-b7de669e2365"

## USA Plot

Plotting daily deaths from COVID-19, as reported by the New York Times (<https://github.com/nytimes/covid-19-data>). The comparison is an aggregate of two CDC categories for US deaths: "Chronic Respiratory Illness" and "Influenza and Pneumonia". Annual numbers in these two categories are added, and divided by 365 for a daily number (see code above). 

Also displayed is a second comparison line, in which "Influenza and Pneumonia" daily rates are calculated based on a 6-month Flu Season (October–March, as the CDC does it).

In [16]:
val deaths: Seq[Int] = totalNewDaily(state = None).map(_.newDeaths).toSeq

val howManyDays: Int = deaths.size
val ipnBase: Seq[Int] = (1 to howManyDays).map( d => usa2017DailyDeathsResp)
val ipnBase6Mo: Seq[Int] = (1 to howManyDays).map( d => usa2017DailyDeathsResp6MoFlu)
val deathBase: Seq[Int] = (1 to howManyDays).map( d => usa2017DailyDeaths)
val cancerBase: Seq[Int] = (1 to howManyDays).map( d => usa2017CancerDaily)
val heartBase: Seq[Int] = (1 to howManyDays).map( d => usa2017HeartDaily)
val accidentBase: Seq[Int] = (1 to howManyDays).map( d => usa2017AccidentDaily)
val suicideBase: Seq[Int] = (1 to howManyDays).map( d => usa2017SuicideDaily)


val trace0 = Scatter(
  (1 to howManyDays),
  deathBase,
  name = "2017 Daily Deaths, All Causes"
)

val trace01 = Scatter(
  (1 to howManyDays),
  cancerBase,
  name = "2017 Daily Deaths, Cancer"
)

val trace02 = Scatter(
  (1 to howManyDays),
  heartBase,
  name = "2017 Daily Deaths, Heart Disease"
)

val trace03 = Scatter(
  (1 to howManyDays),
  accidentBase,
  name = "2017 Daily Deaths, Accidents"
)

val trace04 = Scatter(
  (1 to howManyDays),
  suicideBase,
  name = "2017 Daily Deaths, Suidice"
)

val trace1 = Scatter(
  (1 to howManyDays),
  ipnBase,
  name = "2017 Daily Repiratory-related Deaths"
)

// usa2017DailyDeathsResp6MoFlu

val trace2 = Scatter(
  (1 to howManyDays),
  ipnBase6Mo,
  name = "2017 Daily Repiratory-related Deaths: 6-month Flu Season"

)

val trace3 = Scatter(
  (1 to howManyDays),
  deaths,
  name = "2020 Covid Daily Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(trace0, trace01, trace02, trace03, trace04, trace1, trace2, trace3)

val layout = Layout("Daily USA COVID Deaths vs. Other Causes")

plot(data, layout)

deaths: Seq[Int] = Vector(
  0,
...
howManyDays: Int = 78
ipnBase: Seq[Int] = Vector(
  591,
...
ipnBase6Mo: Seq[Int] = Vector(
  743,
...
deathBase: Seq[Int] = Vector(
  7708,
...
cancerBase: Seq[Int] = Vector(
  1641,
...
heartBase: Seq[Int] = Vector(
  1773,
...
accidentBase: Seq[Int] = Vector(
  465,
...
suicideBase: Seq[Int] = Vector(
  129,
...
trace0: Scatter = Scatter(
  Some(
...
trace01: Scatter = Scatter(
  Some(
...
trace02: Scatter = Scatter(
  Some(
...
trace03: Scatter = Scatter(
  Some(
...
trace04: Scatter = Scatter(
  Some(
...
trace1: Scatter = Scatter(
  Some(
...
trace2: Scatter = Scatter(
  Some(
...
trace3: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Daily USA COVID Deaths vs. Other Causes"),
...
res15_19: String = "plot-4c7295b3-fda4-49ae-9a56-94b826db5364"

## South Carolina

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

Source for 2018 South Carolina Data: [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm) and [SC DHEC](https://www.scdhec.gov/vital-records/parentage/sc-vital-records-data-and-statistics)

In [17]:
// Total Deaths
val sc2018TotalDeaths: Int = 50633
val sc2018DailyDeaths: Int = sc2018TotalDeaths / 365

// Chronic Lower Respiratory Disease
val sc2017TotalDeathsCLRD: Int = 2990
val sc2017DailyDeathsCLRD: Int = 2990 / 365

// Influenza and Pneumonia
val sc2017TotalDeathsIPn: Int = 882
val sc2017DailyDeathsIPn: Int = 882 / 365

// All Respiratory-Related Deaths
val sc2017DailyDeathsAllResp = sc2017DailyDeathsCLRD + sc2017DailyDeathsIPn

// Influenza and Pneumonia: 6 Month Flu Season
val sc2017DailyDeathsAllResp6Mo: Int = {
    sc2017DailyDeathsCLRD + (sc2017TotalDeathsIPn / 182 )
}

sc2018TotalDeaths: Int = 50633
sc2018DailyDeaths: Int = 138
sc2017TotalDeathsCLRD: Int = 2990
sc2017DailyDeathsCLRD: Int = 8
sc2017TotalDeathsIPn: Int = 882
sc2017DailyDeathsIPn: Int = 2
sc2017DailyDeathsAllResp: Int = 10
sc2017DailyDeathsAllResp6Mo: Int = 12

In [18]:
val deaths: Seq[Int] = totalNewDaily(state = Some("South Carolina")).map(_.newDeaths).toSeq

val howManyDays: Int = deaths.size
val ipnBase: Seq[Int] = (1 to howManyDays).map( d => sc2017DailyDeathsAllResp)
val ipnBase6Mo: Seq[Int] = (1 to howManyDays).map( d => sc2017DailyDeathsAllResp6Mo)
val deathBase: Seq[Int] = (1 to howManyDays).map( d => sc2018DailyDeaths)



val trace0 = Scatter(
  (1 to howManyDays),
  deathBase,
  name = "2018 Daily Deaths, All Causes"
)



val trace1 = Scatter(
  (1 to howManyDays),
  ipnBase,
  name = "2018 Daily Repiratory-related Deaths"
)

// usa2017DailyDeathsResp6MoFlu

val trace2 = Scatter(
  (1 to howManyDays),
  ipnBase6Mo,
  name = "2018 Daily Repiratory-related Deaths: 6-month Flu Season"

)

val trace3 = Scatter(
  (1 to howManyDays),
  deaths,
  name = "Covid Daily Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(trace0, trace1, trace2, trace3)

val layout = Layout("South Carolina: Daily COVID Deaths vs. Normal All-Respiratory-Related Deaths")

plot(data, layout)

deaths: Seq[Int] = Vector(
  0,
...
howManyDays: Int = 33
ipnBase: Seq[Int] = Vector(
  10,
...
ipnBase6Mo: Seq[Int] = Vector(
  12,
...
deathBase: Seq[Int] = Vector(
  138,
...
trace0: Scatter = Scatter(
  Some(
...
trace1: Scatter = Scatter(
  Some(
...
trace2: Scatter = Scatter(
  Some(
...
trace3: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some(
...
res17_11: String = "plot-723feaf5-44da-4227-91d8-f8fa12e3c24f"

## USA Case Fatality Rate

A running total of the *total deaths* divided by the *total official cases*.

In [19]:


val groupedDays: Vector[(String, Vector[StateDatum])] = rt.days.groupBy(_.date).toVector.sortBy(_._1)

case class DailyAggregate( cases: Int, deaths: Int)

val pilingUp: Vector[DailyAggregate] = groupedDays.map( d => {
    val dayString: String = d._1
    val dataVec: Vector[StateDatum] = d._2
    val sumCases: Int = dataVec.map(_.cases).sum
    val sumDeaths: Int = dataVec.map(_.deaths).sum
    DailyAggregate(sumCases, sumDeaths)
})

val cfr: Vector[Double] = pilingUp.map(d => {
    d.deaths.toDouble / d.cases.toDouble
})

val cfrSeq: Seq[Double] = cfr.toSeq

val howManyDays: Int = cfrSeq.size


val trace0 = Scatter(
  (1 to howManyDays),
  cfrSeq,
  name = "USA Case Fatality Rate",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(255, 0, 0, 1.0),
    line = Line(
      color = Color.RGBA(255, 0, 0, 1.0),
      width = 0.5
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(trace0)

val layout = Layout("USA Case Fatality Rate")

plot(data, layout)




groupedDays: Vector[(String, Vector[StateDatum])] = Vector(
  ("2020-01-21", Vector(StateDatum("2020-01-21", "Washington", 1, 0))),
...
defined class DailyAggregate
pilingUp: Vector[DailyAggregate] = Vector(
  DailyAggregate(1, 0),
...
cfr: Vector[Double] = Vector(
  0.0,
...
cfrSeq: Seq[Double] = Vector(
  0.0,
...
howManyDays: Int = 78
trace0: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("USA Case Fatality Rate"),
...
res18_9: String = "plot-622b3a65-5f1b-45be-bebc-08f45dbdf838"

## USA Cumulative Numbers

In [20]:
val totalCasesSeq: Seq[Int] = pilingUp.map(_.cases).toSeq

val totalDeathsSeq: Seq[Int] = pilingUp.map(_.deaths).toSeq


val trace0 = Scatter(
  (1 to howManyDays),
  totalCasesSeq,
  name = "Total Cases in USA",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 0, 255, 1.0),
    line = Line(
      color = Color.RGBA(0, 0, 255, 1.0),
      width = 0.5
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val trace1 = Scatter(
  (1 to howManyDays),
  totalDeathsSeq,
  name = "Total Deaths in USA",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(255, 0, 0, 1.0),
    line = Line(
      color = Color.RGBA(255, 0, 0, 1.0),
      width = 0.5
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)



val data = Seq(trace0, trace1)

val layout = Layout("USA Totals")

plot(data, layout)





totalCasesSeq: Seq[Int] = Vector(
  1,
...
totalDeathsSeq: Seq[Int] = Vector(
  0,
...
trace0: Scatter = Scatter(
  Some(
...
trace1: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("USA Totals"),
...
res19_6: String = "plot-20b11754-3405-428e-8030-c9acfb4e7179"

## USA Cumulative Comparison

With 2017 data for comparison, assuming the annual deaths from a given cause are evenly distributed throughout the whole year, how does this grown death-toll compare to COVID19 deaths?

In [21]:
// deaths
// howManyDays
// 
//ipnBase
//deathBase
//cancerBase
//heartBase
//accidentBase
//suicideBase

val deaths: Seq[Int] = totalNewDaily(state = None).map(_.newDeaths).toSeq



val ipnBase: Seq[Int] = (1 to howManyDays).map( d => usa2017DailyDeathsResp)

def dailyToCumulative(v: Seq[Int]):Seq[Int] = {
    v.zipWithIndex.map( t => {
        val d = t._1
        val i = t._2
        if (i == 0 ) {
            d
        }
        else {
            d + (v.take(i+1).sum)
        }
    })
}


val trace1 = Scatter(
  (1 to howManyDays).toSeq,
  dailyToCumulative(ipnBase),
  fill = Fill.ToNextY, 
  name = "2017 Flu & Pneumonia"
)

val trace2 = Scatter(
  (1 to howManyDays).toSeq,
  dailyToCumulative(deaths),
  fill = Fill.ToZeroY,
  name = "2020 COVID"
)

val data = Seq(trace2, trace1)

val layout = Layout("Comparing Growing Death Tolls")

plot(data, layout)




deaths: Seq[Int] = Vector(
  0,
...
ipnBase: Seq[Int] = Vector(
  591,
...
defined function dailyToCumulative
trace1: Scatter = Scatter(
  Some(
...
trace2: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Comparing Growing Death Tolls"),
...
res20_7: String = "plot-52ab3471-4d49-47af-b13e-b9da8d7afcd3"

## Georgia

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

Source for 2017 Georgia Data: [“Stats of the State of South Carolina”, CDC](hhttps://www.cdc.gov/nchs/pressroom/states/georgia/georgia.htm), and [OASIS, Georgia Department of Public Health](https://oasis.state.ga.us/oasis/webquery/qryMortality.aspx).

In [22]:
// Total Deaths
val ga2018TotalDeaths: Int = 83122
val ga2018DailyDeaths: Int = ga2018TotalDeaths / 365



// All Respiratory-Related Deaths
val ga2017DailyDeathsAllResp = 8526 / 365


ga2018TotalDeaths: Int = 83122
ga2018DailyDeaths: Int = 227
ga2017DailyDeathsAllResp: Int = 23

In [23]:
val gaDeaths: Seq[Int] = totalNewDaily(state = Some("Georgia")).map(_.newDeaths).toSeq

val howManyDays: Int = deaths.size
val gaAllDeathsBase: Seq[Int] = (1 to howManyDays).map( d => ga2018DailyDeaths)
val gaRespDeaths: Seq[Int] = (1 to howManyDays).map( d => ga2017DailyDeathsAllResp)



val trace0 = Scatter(
  (1 to howManyDays),
  gaAllDeathsBase,
  name = "2017 Daily Deaths, All Causes"
)



val trace1 = Scatter(
  (1 to howManyDays),
  gaRespDeaths,
  name = "2017 Daily Deaths, All Respiratory Disease"
)



val trace3 = Scatter(
  (1 to howManyDays),
  gaDeaths,
  name = "Covid Daily Deaths",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(trace0, trace1, trace3)

val layout = Layout("Georgia: Daily COVID Deaths vs. Normal All-Respiratory-Related Deaths")

plot(data, layout)

gaDeaths: Seq[Int] = Vector(
  0,
...
howManyDays: Int = 78
gaAllDeathsBase: Seq[Int] = Vector(
  227,
...
gaRespDeaths: Seq[Int] = Vector(
  23,
...
trace0: Scatter = Scatter(
  Some(
...
trace1: Scatter = Scatter(
  Some(
...
trace3: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Georgia: Daily COVID Deaths vs. Normal All-Respiratory-Related Deaths"),
...
res22_9: String = "plot-a45b88de-87a0-469c-b285-9cdf9b9044f9"

## England & Wales Deaths

Source for Data:[The Office of National Statistics, UK](https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/weeklyprovisionalfiguresondeathsregisteredinenglandandwales)

In [24]:
val weeks = Vector("3-Jan-20","10-Jan-20","17-Jan-20","24-Jan-20","31-Jan-20","7-Feb-20","14-Feb-20","21-Feb-20","28-Feb-20","6-Mar-20","13-Mar-20","20-Mar-20","27-Mar-20")
val ukDeath = Vector(12254,14058,12990,11856,11612,10986,10944,10841,10816,10895,11019,10645,11141)
val ukDeaths5yrs = Vector(12175,13822,13216,12760,12206,11925,11627,11548,11183,11498,11205,10573,10130)
val uk2018RespDeaths = Vector(2361,3075,2829,2672,2479,2197,2081,2128,1900,2302,2271,1946,1657)
val uk2020RespDeaths = Vector(2141,2477,2188,1893,1746,1572,1602,1619,1546,1557,1488,1514,1534)
val ukCovidDeaths = Vector(0,0,0,0,0,0,0,0,0,0,5,103,539)


weeks: Vector[String] = Vector(
  "3-Jan-20",
...
ukDeath: Vector[Int] = Vector(
  12254,
...
ukDeaths5yrs: Vector[Int] = Vector(
  12175,
...
uk2018RespDeaths: Vector[Int] = Vector(
  2361,
...
uk2020RespDeaths: Vector[Int] = Vector(
  2141,
...
ukCovidDeaths: Vector[Int] = Vector(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 103, 539)

In [25]:
val total2020Uk = ukDeath.sum
val aveTotalBase = ukDeaths5yrs.sum

// a positive number means more deaths this year
val moreDeaths = total2020Uk - aveTotalBase 
val percentMoreTotal = (( moreDeaths.toDouble / aveTotalBase.toDouble ) * 100).toInt

val total2020Resp = uk2018RespDeaths.sum
val total2018Resp = uk2020RespDeaths.sum

// a positive number means more deaths this year
val moreRespDeaths = total2020Resp - total2018Resp
val percentMoreResp = (( moreRespDeaths.toDouble / total2018Resp.toDouble ) * 100).toInt

total2020Uk: Int = 150057
aveTotalBase: Int = 153868
moreDeaths: Int = -3811
percentMoreTotal: Int = -2
total2020Resp: Int = 29898
total2018Resp: Int = 22877
moreRespDeaths: Int = 7021
percentMoreResp: Int = 30

In [26]:
val cumuUkDeathsByMonth2020: Vector[Int] = {
    ukDeath.zipWithIndex.map( t => {
        val d = t._1
        val i = t._2
        if (i == 0 ) {
            d
        }
        else {
            d + (ukDeath.take(i+1).sum)
        }
    })
}

val cumuUkDeathsByMonthAve: Vector[Int] = {
    ukDeaths5yrs.zipWithIndex.map( t => {
        val d = t._1
        val i = t._2
        if (i == 0 ) {
            d
        }
        else {
            d + (ukDeaths5yrs.take(i+1).sum)
        }
    })
}

val trace1 = Scatter(
  (1 to cumuUkDeathsByMonth2020.size).toSeq,
  cumuUkDeathsByMonth2020.toSeq,
  fill = Fill.ToZeroY, 
  name = "2020 Weekly All Deaths"
)

val trace2 = Scatter(
  (1 to cumuUkDeathsByMonth2020.size).toSeq,
  cumuUkDeathsByMonthAve.toSeq,
  fill = Fill.ToNextY,
  name = "2015-2019 Weekly Ave. All Deaths"
)

val data = Seq(trace1, trace2)

val layout = Layout("England and Wales: Comparative Deaths, Week-by-Week")

plot(data, layout)



cumuUkDeathsByMonth2020: Vector[Int] = Vector(
  12254,
...
cumuUkDeathsByMonthAve: Vector[Int] = Vector(
  12175,
...
trace1: Scatter = Scatter(
  Some(
...
trace2: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("England and Wales: Comparative Deaths, Week-by-Week"),
...
res25_6: String = "plot-9715da1e-81c3-4a8b-a8f9-e0466c018a4b"